In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from expipe_plugin_cinpla.widgets import browser
import expipe

In [3]:
browser.display('D:/projects/novel-env')

Invoking remote shell
Initializing transfer of "D:\projects\novel-env\actions\142-200521-2\data\main.exdir\acquisition\2" to "193.157.210.157"
mkdir /tmp/process_9274270
end of stdout buffer. finished with exit status
Packing tar archive
D:\projects\novel-env\actions\142-200521-2\data\main.exdir\acquisition\2.tar


Transferring: b'2.tar':  25%|#############7                                        | 358M/1.41G [00:30<01:29, 11.8MB/s]

In [4]:
import spikeextractors as se
import spiketoolkit as st
from pathlib import Path
import time

In [8]:
string = f"{openephys_path}"

In [9]:
00

0

In [10]:
probe_string = "C:\apps\open-ephys\tetrode_32_openephys.prb"

In [19]:
openephys_path = Path("D:/Data/142_2021-05-19_12-53-50_3")
probe_path = "C:/apps/open-ephys/tetrode_32_openephys.prb"

In [20]:
probe_path

'C:/apps/open-ephys/tetrode_32_openephys.prb'

In [21]:
spikesort=True
compute_lfp=True
compute_mua=False
parallel=False
spikesorter_params=None
server=None
bad_channels=[]
ref='cmr'
split="half"

In [22]:
recording = se.OpenEphysRecordingExtractor(str(openephys_path))
recording = recording.load_probe_file(probe_path)

if 'auto' not in bad_channels and len(bad_channels) > 0:
    recording_active = st.preprocessing.remove_bad_channels(recording, bad_channel_ids=bad_channels)
else:
    recording_active = recording

# apply filtering and cmr
print('Writing filtered and common referenced data')

tmp_folder = Path(f"tmp_test_si")

freq_min_hp = 300
freq_max_hp = 3000
freq_min_lfp = 1
freq_max_lfp = 300
freq_resample_lfp = 1000
freq_resample_mua = 1000
type_hp = 'butter'
order_hp = 5

recording_hp = st.preprocessing.bandpass_filter(
    recording_active, freq_min=freq_min_hp, freq_max=freq_max_hp,
    filter_type=type_hp, order=order_hp)

if ref is not None:
    if ref.lower() == 'cmr':
        reference = 'median'
    elif ref.lower() == 'car':
        reference = 'average'
    else:
        raise Exception("'reference' can be either 'cmr' or 'car'")
    if split == 'all':
        recording_cmr = st.preprocessing.common_reference(recording_hp, reference=reference)
    elif split == 'half':
        groups = [recording.get_channel_ids()[:int(len(recording.get_channel_ids()) / 2)],
                  recording.get_channel_ids()[int(len(recording.get_channel_ids()) / 2):]]
        recording_cmr = st.preprocessing.common_reference(recording_hp, groups=groups, reference=reference)
    else:
        if isinstance(split, list):
            recording_cmr = st.preprocessing.common_reference(recording_hp, groups=split, reference=reference)
        else:
            raise Exception("'split' must be a list of lists")
else:
    recording_cmr = recording

if 'auto' in bad_channels:
    recording_cmr = st.preprocessing.remove_bad_channels(recording_cmr, bad_channel_ids=None,
                                                         bad_threshold=bad_threshold, seconds=10)
    recording_active = se.SubRecordingExtractor(
        recording, channel_ids=recording_cmr.active_channels)

print("Active channels: ", len(recording_active.get_channel_ids()))
recording_lfp = st.preprocessing.bandpass_filter(
    recording_active, freq_min=freq_min_lfp, freq_max=freq_max_lfp)
recording_lfp = st.preprocessing.resample(
    recording_lfp, freq_resample_lfp)
recording_mua = st.preprocessing.resample(
    st.preprocessing.rectify(recording_active), freq_resample_mua)

if spikesort:
    print('Bandpass filter')
    t_start = time.time()
    recording_cmr = se.CacheRecordingExtractor(recording_cmr, save_path=tmp_folder / 'filt.dat')
    print('Filter time: ', time.time() - t_start)
if compute_lfp:
    print('Computing LFP')
    t_start = time.time()
    recording_lfp = se.CacheRecordingExtractor(recording_lfp, save_path=tmp_folder / 'lfp.dat')
    print('Filter time: ', time.time() - t_start)

if compute_mua:
    print('Computing MUA')
    t_start = time.time()
    recording_mua = se.CacheRecordingExtractor(recording_mua, save_path=tmp_folder / 'mua.dat')
    print('Filter time: ', time.time() - t_start)


Loading Open-Ephys: reading settings...
Decoding data from  binary  format
Reading oebin file
Writing filtered and common referenced data
Active channels:  32
Bandpass filter
Filter time:  222.15481424331665
Computing LFP
Filter time:  247.83862447738647
